In [1]:
pip install ftfy regex tqdm


In [2]:
pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-93izdjom
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-93izdjom
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 767.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.0 MB/s eta 0

In [3]:
import torch
import clip
from PIL import Image


In [4]:
# Load the CLIP model and preprocessor
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 135MiB/s]


In [5]:
# List of sentences for each organ
sentences = {
    'Myocardium':  ["Myocardium",
    "The myocardium is the middle layer of the heart's walls, consisting of cardiac muscle tissue",
    "It gives the heart its characteristic striated appearance when viewed microscopically",
    "In scans, the myocardium is not directly visible but plays a crucial role in heart function",
    "It occupies the entire heart wall and is responsible for the heart's contractions",
    "Its structure includes interconnected fibers that facilitate synchronized contractions",
    "Microscopically, the myocardium's cross-striations and branching fibers differentiate it from other tissues",
    "Echocardiography and MRI can indirectly assess myocardial health and thickness",
    "The myocardium's texture and thickness can vary based on heart rate and physical condition",
    "During stress tests, changes in myocardial perfusion are indicative of potential heart problems",
    "Cardiac arrhythmias and ischemic conditions can affect the myocardium's appearance and function in scans"
    ],

    'Left Ventricle':["Left Ventricle",
    "The left ventricle is a thick-walled, cone-shaped chamber located at the bottom of the heart",
    "Its muscular walls give it a distinct appearance and the ability to generate strong contractions",
    "In scans, the left ventricle is easily visible due to its significant role in pumping oxygen-rich blood to the body",
    "The left ventricle occupies a substantial portion of the heart's left side and forms the apex of the heart",
    "Its shape resembles a rounded cone, broader at the top and tapering towards the apex",
    "The left ventricle's strong contractions contribute to its robust physical appearance",
    "In contrast-enhanced scans, the left ventricle's internal structures, including the papillary muscles and chordae tendineae, are more distinguishable",
    "Its location near the apex and its larger size set it apart from neighboring heart chambers",
    "The left ventricle's rhythmic contractions can be detected through echocardiography, aiding in its identification",
    "Abnormalities in left ventricular function can impact the heart's pumping efficiency and are visible in various imaging modalities"
    ],

    'Right Ventricle':  ["Right Ventricle",
    "The right ventricle is a thinner-walled chamber located on the heart's right side, above the left ventricle",
    "Its crescent-shaped appearance is adapted for pumping blood to the lungs",
    "In scans, the right ventricle is clearly visible, especially in views of the heart's frontal section",
    "The right ventricle occupies a substantial part of the heart's anterior surface",
    "Its shape resembles a crescent, curving around the left ventricle and forming the heart's base",
    "The right ventricle's thinner walls contribute to its more delicate physical appearance",
    "In imaging, the tricuspid valve and pulmonary valve provide distinguishing landmarks for the right ventricle",
    "Its proximity to the sternum helps differentiate the right ventricle from the left ventricle",
    "Echocardiography can visualize the right ventricle's movements and detect potential anomalies",
    "Right ventricular hypertrophy can change the chamber's shape and visibility in imaging studies"
    ]
}

# Create lists to store the mean text embeddings for each organ
organ_vectors =[]

# Process sentences and calculate mean text embeddings for each organ
for organ, organ_sentences in sentences.items():
    text_input = torch.cat([clip.tokenize(sent) for sent in organ_sentences]).to(device)

    # Get the text embeddings
    with torch.no_grad():
        text_features = model.encode_text(text_input)

    # Take the mean of the text embeddings (num_sentences, 512) along the first dimension (rows)
    mean_text_features = torch.mean(text_features, dim=0)

#     # Append the mean_text_features to the corresponding list
    organ_vectors.append(mean_text_features)

# Save the vectors to a .pt file
output_path = "card_11clip-MLR.pt"
torch.save(organ_vectors, output_path)


